<a href="https://colab.research.google.com/github/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/blob/main/7%20Feature%20selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Feature Engineering & Feature selection**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install openpyxl -q
import openpyxl
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# for grid
!pip install geopy -q
from geopy.distance import geodesic

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Now CatBoost
!pip install catboost -q
from catboost import CatBoostRegressor, Pool
# Now XGBoost
import xgboost as xgb

# Set display options
%matplotlib inline
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
# apartments = pd.read_excel('https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/apartments_5.xlsx', engine='openpyxl')

In [4]:
# import pickle

!wget https://github.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/raw/main/apartments5.pkl -q

with open('apartments5.pkl', 'rb') as f:
    apartments = pickle.load(f)

!wget https://github.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/raw/main/lat_bin.pkl -q
!wget https://github.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/raw/main/lon_bin.pkl -q

with open('lat_bin.pkl', 'rb') as f:
    lat_bin = pickle.load(f)

with open('lon_bin.pkl', 'rb') as f:
    lon_bin = pickle.load(f)


In [4]:
apartments.shape

(19664, 44)

In [5]:
apartments.drop(columns=['Host Since','First Review', 'Last Review','Reviews', 'Price Log', 'Latitude_Bin', 'Longitude_Bin' ], inplace=True)

In [6]:
apartments['Is Superhost'] = apartments['Is Superhost'].replace({'t': 1, 'f': 0}).fillna(0).astype(int)
apartments['Instant Bookable'] = apartments['Instant Bookable'].replace({'t': 1, 'f': 0}).fillna(0).astype(int)
apartments['Is Exact Location'] = apartments['Is Exact Location'].replace({'t': 1, 'f': 0}).fillna(0).astype(int)
apartments['Property Type'] = apartments['Property Type'].replace({'Apartment': 1, 'Condominium': 0}).fillna(0).astype(int)
apartments['Room Type'] = apartments['Room Type'].replace({'Private room': 1, 'Entire home/apt': 0}).fillna(0).astype(int)

apartments['Host Response Time'] = apartments['Host Response Time'].replace(
    {'within an hour':0, 'within a few hours': 1, 'within a day': 2, 'a few days or more': 3 }).fillna(0).astype(int)

In [20]:
categorical_cols = apartments.select_dtypes(exclude='number').columns
for col in categorical_cols:
  print(f"Value counts for column '{col}' :")
  print(apartments[col].value_counts())
  print(f"NaN values: {apartments[col].isnull().sum()} ")
  print()
  print()

Value counts for column 'Listing ID' :
Listing ID
10002699    1
30934593    1
30947505    1
30946969    1
30946705    1
           ..
20716366    1
20716294    1
20710491    1
20710283    1
9996186     1
Name: count, Length: 19664, dtype: int64
NaN values: 0 


Value counts for column 'neighbourhood' :
neighbourhood
NeukÃ¶lln                3182
Prenzlauer Berg          2550
Kreuzberg                2509
Friedrichshain           2460
Mitte                    1749
Wedding                  1316
SchÃ¶neberg               935
Moabit                    715
Charlottenburg            696
Wilmersdorf               400
Pankow                    272
Rummelsburg               223
Tempelhof                 220
WeiÃensee                181
Lichtenberg               166
Alt-Treptow               156
Tiergarten                153
Steglitz                  135
Reinickendorf             133
Westend                    89
Lichterfelde               89
Friedenau                  77
Baumschulenweg        

In [23]:
# prompt: значения 'neighbourhood' по которым менее 10 наблюдений заменить на "other"

# neighbourhood_counts = apartments['neighbourhood'].value_counts()
# apartments['neighbourhood'] = np.where(apartments['neighbourhood'].isin(neighbourhood_counts[neighbourhood_counts >= 10].index), apartments['neighbourhood'], 'Other')

# Postal_counts = apartments['Postal Code'].value_counts()
# apartments['Postal Code'] = np.where(apartments['Postal Code'].isin(Postal_counts[Postal_counts >= 10].index), apartments['Postal Code'], 'Other')
# Postal_counts

,count
Postal Code,
10245.0,855
10247.0,749
10437.0,658
10115.0,580
10999.0,542
10405.0,517
10997.0,517
10439.0,507
10119.0,485


# CatBoost feature importance

for CatBost no nessery

In [25]:
# X[['Overall Rating', 'Accuracy Rating',  'Cleanliness Rating', 'Checkin Rating',
#    'Communication Rating',  'Location Rating', 'Value Rating', 'Polarity_min',
#    'Polarity_max', 'Polarity_median', 'Polarity_mean']] = \
# X[['Overall Rating', 'Accuracy Rating',  'Cleanliness Rating', 'Checkin Rating',
#    'Communication Rating',  'Location Rating', 'Value Rating', 'Polarity_min',
#    'Polarity_max', 'Polarity_median', 'Polarity_mean']].fillna(-2)


In [7]:
X = apartments.drop(['Listing ID', 'Price'], axis=1)
y = apartments['Price']

In [9]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# !pip install catboost -q
# from catboost import CatBoostRegressor, Pool  # Now CatBoost is available

num_features = X.select_dtypes(include='number').columns
cat_features = X.select_dtypes(exclude='number').columns

for col in cat_features:
    if X[col].dtype != 'str':
        X[col] = X[col].astype(str)

# Разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Convert cat_features to a list
cat_features = cat_features.tolist()  # This is the key change

# Создаем модели для обучающей и тестовой выборок с указанием категориальных признаков
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

# Инициализируем и обучаем модель
model = CatBoostRegressor(iterations=2000, learning_rate=0.02, depth=6, verbose=100)
model.fit(train_pool)

# Предсказания на обучающей и тестовой выборках
y_train_pred = model.predict(train_pool)
y_test_pred = model.predict(test_pool)

# Вычисляем метрики для обучающей выборки
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Вычисляем метрики для тестовой выборки
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Выводим метрики на обучающей выборке
print("Metrics on Training Set:")
print(f"Mean Squared Error: {train_mse}")
print(f"Mean Absolute Error: {train_mae}")
print(f"R^2 Score: {train_r2}")

# Выводим метрики на тестовой выборке
print("\nMetrics on Test Set:")
print(f"Mean Squared Error: {test_mse}")
print(f"Mean Absolute Error: {test_mae}")
print(f"R^2 Score: {test_r2}")


# Выводим важность признаков
feature_importances = model.get_feature_importance(train_pool)
# Create a list of tuples (feature_name, importance)
feature_importance_tuples = []
for feature_index in range(len(feature_importances)):
    if feature_index < len(num_features):
        feature_name = num_features[feature_index]
    else:
        feature_name = cat_features[feature_index - len(num_features)]
    importance = feature_importances[feature_index]
    feature_importance_tuples.append((feature_name, importance))

# Sort the list of tuples by importance in descending order
feature_importance_tuples.sort(key=lambda x: x[1], reverse=True)

# Print the sorted feature importances
for feature_name, importance in feature_importance_tuples:
    print(f'Feature: {feature_name}, Importance: {importance}')


0:	learn: 38.3131080	total: 20.7ms	remaining: 1m 1s
100:	learn: 26.3409174	total: 2.77s	remaining: 1m 19s
200:	learn: 24.9802118	total: 4.81s	remaining: 1m 7s
300:	learn: 24.4332915	total: 6.26s	remaining: 56.2s
400:	learn: 24.0592427	total: 7.73s	remaining: 50.1s
500:	learn: 23.7078781	total: 10.9s	remaining: 54.3s
600:	learn: 23.3451332	total: 12.4s	remaining: 49.5s
700:	learn: 23.0193100	total: 13.9s	remaining: 45.6s
800:	learn: 22.7442809	total: 15.4s	remaining: 42.4s
900:	learn: 22.5048215	total: 16.9s	remaining: 39.4s
1000:	learn: 22.2913647	total: 18.3s	remaining: 36.6s
1100:	learn: 22.0640485	total: 19.9s	remaining: 34.3s
1200:	learn: 21.8749419	total: 22.2s	remaining: 33.3s
1300:	learn: 21.7064044	total: 24.6s	remaining: 32.1s
1400:	learn: 21.5334507	total: 26.1s	remaining: 29.8s
1500:	learn: 21.3703779	total: 27.6s	remaining: 27.5s
1600:	learn: 21.2107627	total: 29s	remaining: 25.4s
1700:	learn: 21.0626685	total: 30.5s	remaining: 23.3s
1800:	learn: 20.9130849	total: 32s	remai

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df = apartments.drop(['Listing ID'], axis=1)

# Выделяем категориальные признаки
categorical_features = df.select_dtypes(exclude='number').columns


# Закодируем категориальные переменные
le = LabelEncoder()
for feature in categorical_features:
    df[feature] = le.fit_transform(df[feature])

# Разделим признаки и целевую переменную
X = df.drop('Price', axis=1)
y = df['Price']

# Разделим данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создадим объект модели
model = xgb.XGBRegressor(n_estimators=350, learning_rate=0.02, max_depth=5, verbosity=1)


# Определим eval_set для контроля обучения
eval_set = [(X_train, y_train), (X_test, y_test)]

# Обучим модель с выводом RMSE на каждой итерации, убираем evals_result
model.fit(X_train, y_train, eval_set=eval_set, verbose=True)

# # Обучим модель
# model.fit(X_train, y_train)

# Предсказания на тренировочной и тестовой выборках
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Вывод метрик
print("Training MSE:", mean_squared_error(y_train, y_train_pred))
print("Test MSE:", mean_squared_error(y_test, y_test_pred))

print("Training MAE:", mean_absolute_error(y_train, y_train_pred))
print("Test MAE:", mean_absolute_error(y_test, y_test_pred))

print("Training R^2:", r2_score(y_train, y_train_pred))
print("Test R^2:", r2_score(y_test, y_test_pred))

# Важность признаков
importance = model.get_booster().get_score(importance_type='weight')
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

# Вывод важности признаков в порядке убывания
print("\nFeature importance in descending order:")
for feature, score in sorted_importance:
    print(f"{feature}: {score}")


[0]	validation_0-rmse:38.38647	validation_1-rmse:37.72816
[1]	validation_0-rmse:38.00316	validation_1-rmse:37.36335
[2]	validation_0-rmse:37.63117	validation_1-rmse:37.01159
[3]	validation_0-rmse:37.27055	validation_1-rmse:36.67115
[4]	validation_0-rmse:36.91912	validation_1-rmse:36.34008
[5]	validation_0-rmse:36.57919	validation_1-rmse:36.02022
[6]	validation_0-rmse:36.24933	validation_1-rmse:35.71102
[7]	validation_0-rmse:35.92068	validation_1-rmse:35.40102
[8]	validation_0-rmse:35.60744	validation_1-rmse:35.10763
[9]	validation_0-rmse:35.29751	validation_1-rmse:34.81637
[10]	validation_0-rmse:35.00204	validation_1-rmse:34.53853
[11]	validation_0-rmse:34.71375	validation_1-rmse:34.27245
[12]	validation_0-rmse:34.42913	validation_1-rmse:34.00806
[13]	validation_0-rmse:34.15343	validation_1-rmse:33.75193
[14]	validation_0-rmse:33.88949	validation_1-rmse:33.50768
[15]	validation_0-rmse:33.62931	validation_1-rmse:33.26859
[16]	validation_0-rmse:33.37938	validation_1-rmse:33.04111
[17]	va

In [33]:
#!pip install --upgrade xgboost
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Создадим пример данных
data = {
    'Age': [25, 32, 47, 51, 62, 28, 45, 52, 23, 39],
    'Salary': [50000, 60000, 150000, 200000, 120000, 55000, 95000, 105000, 40000, 80000],
    'Department': ['HR', 'Finance', 'IT', 'IT', 'HR', 'Finance', 'Finance', 'HR', 'IT', 'HR'],
    'House Price': [200000, 250000, 550000, 650000, 500000, 220000, 430000, 480000, 180000, 330000]
}

df = pd.DataFrame(data)

# Выделяем категориальные признаки
categorical_features = ['Department']

# Закодируем категориальные переменные
le = LabelEncoder()
for feature in categorical_features:
    df[feature] = le.fit_transform(df[feature])

# Разделим признаки и целевую переменную
X = df.drop('House Price', axis=1)
y = df['House Price']

# Разделим данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создадим объект модели
model = xgb.XGBRegressor()

# Определим eval_set для контроля обучения
eval_set = [(X_train, y_train), (X_test, y_test)]

# Определим eval_set для контроля обучения
eval_set = [(X_train, y_train), (X_test, y_test)]

# Обучим модель с выводом RMSE на каждой итерации, убираем evals_result
model.fit(X_train, y_train, eval_set=eval_set, verbose=True)

# Предсказания на тренировочной и тестовой выборках
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Вывод метрик
print("Training MSE:", mean_squared_error(y_train, y_train_pred))
print("Test MSE:", mean_squared_error(y_test, y_test_pred))

print("Training MAE:", mean_absolute_error(y_train, y_train_pred))
print("Test MAE:", mean_absolute_error(y_test, y_test_pred))

print("Training R^2:", r2_score(y_train, y_train_pred))
print("Test R^2:", r2_score(y_test, y_test_pred))

# Важность признаков
importance = model.get_booster().get_score(importance_type='weight')
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

# Вывод важности признаков в порядке убывания
print("\nFeature importance in descending order:")
for feature, score in sorted_importance:
    print(f"{feature}: {score}")


[0]	validation_0-rmse:119882.46331	validation_1-rmse:170383.57254
[1]	validation_0-rmse:97266.01514	validation_1-rmse:136948.01817
[2]	validation_0-rmse:79433.52905	validation_1-rmse:110603.82576
[3]	validation_0-rmse:65477.96701	validation_1-rmse:90018.77564
[4]	validation_0-rmse:54004.14533	validation_1-rmse:74131.41822
[5]	validation_0-rmse:44694.34654	validation_1-rmse:63322.36963
[6]	validation_0-rmse:37029.85122	validation_1-rmse:54072.07808
[7]	validation_0-rmse:30783.09749	validation_1-rmse:47405.47235
[8]	validation_0-rmse:25688.51673	validation_1-rmse:42767.99135
[9]	validation_0-rmse:21533.45285	validation_1-rmse:39664.83833
[10]	validation_0-rmse:18147.33580	validation_1-rmse:37670.29302
[11]	validation_0-rmse:15393.10282	validation_1-rmse:36440.42020
[12]	validation_0-rmse:13049.67223	validation_1-rmse:34519.98470
[13]	validation_0-rmse:11067.16963	validation_1-rmse:32961.24172
[14]	validation_0-rmse:9389.77571	validation_1-rmse:31694.12311
[15]	validation_0-rmse:7968.8979

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
!pip install category_encoders
from category_encoders import TargetEncoder

# Создадим пример данных
data = {
    'Age': [25, 32, 47, 51, 62, 28, 45, 52, 23, 39],
    'Salary': [50000, 60000, 150000, 200000, 120000, 55000, 95000, 105000, 40000, 80000],
    'Department': ['HR', 'Finance', 'IT', 'IT', 'HR', 'Finance', 'Finance', 'HR', 'IT', 'HR'],
    'House Price': [200000, 250000, 550000, 650000, 500000, 220000, 430000, 480000, 180000, 330000]
}

df = pd.DataFrame(data)

# Разделим признаки и целевую переменную
X = df.drop('House Price', axis=1)
y = df['House Price']

# Разделим данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применим таргет энкодинг только к тренировочной выборке
target_encoder = TargetEncoder()
X_train_encoded = X_train.copy()
X_train_encoded['Department'] = target_encoder.fit_transform(X_train['Department'], y_train)

# Применим таргет энкодинг к тестовой выборке с использованием тех же параметров
X_test_encoded = X_test.copy()
X_test_encoded['Department'] = target_encoder.transform(X_test['Department'])

# Создадим объект модели
model = xgb.XGBRegressor()

# Определим eval_set для контроля обучения
eval_set = [(X_train_encoded, y_train), (X_test_encoded, y_test)]

# Обучим модель с ранней остановкой
# Обучим модель с выводом RMSE на каждой итерации, убираем evals_result
model.fit(X_train, y_train, eval_set=eval_set, verbose=True)

# Предсказания на тренировочной и тестовой выборках
y_train_pred = model.predict(X_train_encoded)
y_test_pred = model.predict(X_test_encoded)

# Вывод метрик
print("Training MSE:", mean_squared_error(y_train, y_train_pred))
print("Test MSE:", mean_squared_error(y_test, y_test_pred))

print("Training MAE:", mean_absolute_error(y_train, y_train_pred))
print("Test MAE:", mean_absolute_error(y_test, y_test_pred))

print("Training R^2:", r2_score(y_train, y_train_pred))
print("Test R^2:", r2_score(y_test, y_test_pred))

# Важность признаков
importance = model.get_booster().get_score(importance_type='weight')
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

# Вывод важности признаков в порядке убывания
print("\nFeature importance in descending order:")
for feature, score in sorted_importance:
    print(f"{feature}: {score}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.6 MB/s eta 0:00:00


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Department: object

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
!pip install category_encoders
from category_encoders import TargetEncoder

# Создадим пример данных
data = {
    'Age': [25, 32, 47, 51, 62, 28, 45, 52, 23, 39],
    'Salary': [50000, 60000, 150000, 200000, 120000, 55000, 95000, 105000, 40000, 80000],
    'Department': ['HR', 'Finance', 'IT', 'IT', 'HR', 'Finance', 'Finance', 'HR', 'IT', 'HR'],
    'House Price': [200000, 250000, 550000, 650000, 500000, 220000, 430000, 480000, 180000, 330000]
}

df = pd.DataFrame(data)

# Разделим признаки и целевую переменную
X = df.drop('House Price', axis=1)
y = df['House Price']

# Разделим данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применим таргет энкодинг к тренировочной и тестовой выборке
target_encoder = TargetEncoder()
X_train_encoded = X_train.copy()
X_train_encoded['Department'] = target_encoder.fit_transform(X_train['Department'], y_train)

X_test_encoded = X_test.copy()
X_test_encoded['Department'] = target_encoder.transform(X_test['Department']) # Use the same encoder fitted on training data

# Создадим объект модели
model = xgb.XGBRegressor()

# Определим eval_set для контроля обучения
eval_set = [(X_train_encoded, y_train), (X_test_encoded, y_test)] # Use encoded data for evaluation

# Обучим модель с выводом RMSE на каждой итерации
model.fit(X_train_encoded, y_train, eval_set=eval_set, verbose=True) # Fit with encoded data

# Предсказания на тренировочной и тестовой выборках
y_train_pred = model.predict(X_train_encoded) # Predict using encoded data
y_test_pred = model.predict(X_test_encoded) # Predict using encoded data

# Вывод метрик
print("Training MSE:", mean_squared_error(y_train, y_train_pred))
print("Test MSE:", mean_squared_error(y_test, y_test_pred))

print("Training MAE:", mean_absolute_error(y_train, y_train_pred))
print("Test MAE:", mean_absolute_error(y_test, y_test_pred))

print("Training R^2:", r2_score(y_train, y_train_pred))
print("Test R^2:", r2_score(y_test, y_test_pred))

# Важность признаков
importance = model.get_booster().get_score(importance_type='weight')
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

# Вывод важности признаков в порядке убывания
print("\nFeature importance in descending order:")
for feature, score in sorted_importance:
    print(f"{feature}: {score}")

[0]	validation_0-rmse:119882.46331	validation_1-rmse:170383.57254
[1]	validation_0-rmse:97266.01514	validation_1-rmse:136948.01817
[2]	validation_0-rmse:79433.52905	validation_1-rmse:110603.82576
[3]	validation_0-rmse:65477.96701	validation_1-rmse:90018.77564
[4]	validation_0-rmse:54004.14533	validation_1-rmse:74131.41822
[5]	validation_0-rmse:44694.34654	validation_1-rmse:63322.36963
[6]	validation_0-rmse:37029.85122	validation_1-rmse:54072.07808
[7]	validation_0-rmse:30783.09749	validation_1-rmse:47405.47235
[8]	validation_0-rmse:25688.51673	validation_1-rmse:42767.99135
[9]	validation_0-rmse:21533.45285	validation_1-rmse:39664.83833
[10]	validation_0-rmse:18147.33580	validation_1-rmse:37670.29302
[11]	validation_0-rmse:15393.10282	validation_1-rmse:36440.42020
[12]	validation_0-rmse:13049.67223	validation_1-rmse:34519.98470
[13]	validation_0-rmse:11067.16963	validation_1-rmse:32961.24172
[14]	validation_0-rmse:9389.77571	validation_1-rmse:31694.12311
[15]	validation_0-rmse:7968.8979

# Encoding

Converting categorical data to numerical

In [ ]:
categorical_cols = apartments.select_dtypes(exclude='number').columns
for col in categorical_cols:
  print(f"Value counts for column '{col}' :")
  print(apartments[col].value_counts())
  print(f"NaN values: {apartments[col].isnull().sum()} ")
  print()
  print()

In [ ]:
# Perform one-hot encoding and drop the first column
neighborhood_dummies = pd.get_dummies(apartments['Neighborhood Group'], prefix='Neighborhood', drop_first=True)

# Concatenate the dummy variables with the original DataFrame
apartments = pd.concat([apartments, neighborhood_dummies], axis=1)

In [ ]:
numeric_columns = apartments.select_dtypes(include=['number']).columns

print(numeric_columns)

Index(['Host Response Time', 'Host Response Rate', 'Is Superhost', 'Latitude',
       'Longitude', 'Is Exact Location', 'Property Type', 'Room Type',
       'Accomodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Price',
       'Guests Included', 'Min Nights', 'Reviews', 'Overall Rating',
       'Accuracy Rating', 'Cleanliness Rating', 'Checkin Rating',
       'Communication Rating', 'Location Rating', 'Value Rating',
       'Instant Bookable', 'Polarity_min', 'Polarity_max', 'Polarity_median',
       'Polarity_mean', 'Host Since Years', 'Last Review Years',
       'First Review Years', 'Latitude_Bin', 'Longitude_Bin', 'Price Log',
       'Reviews Log', 'Distance_from_center'],
      dtype='object')


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

numeric_columns = numeric_columns.drop(['Price Log'])

df = apartments[numeric_columns].copy()

# Предположим, что df - ваш датафрейм
X = df.drop('Price', axis=1)
y = df['Price']

# Создаем модель Лассо
model = Lasso(alpha=0.1)  # Параметр alpha регулирует степень регуляризации
model.fit(X, y)
y_pred = model.predict(X)

r2 = r2_score(y, y_pred)
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = mean_squared_error(y, y_pred, squared=False)

print(f"\n Lasso")
print(f"R^2: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")


# Важность признаков
feature_importance = pd.Series(model.coef_, index=X.columns).sort_values(ascending=False)
print("Feature importance using Lasso:")
print(feature_importance)

ValueError: Input X contains NaN.
Lasso does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
#apartments.to_excel('apartments_6.xlsx', index=False)

# import pickle
with open('apartments6.pkl', 'wb') as f:
    pickle.dump(apartments, f)


# Average of neighbors calculation

In [ ]:
def weighted_location_rating(lat, lon, lat_bin, lon_bin, df, target_col, weight_col, primary_threshold_m=250,
                             primary_bin_range = 1,  secondary_threshold_m=500, secondary_bin_range = 2):

    epsilon = 0.03  # Маленькое значение для предотвращения деления на ноль
    #  determine_bins(lat, lon, lat_bin, lon_bin)
    lat_bin = np.digitize(lat, lat_bin) - 1
    lon_bin = np.digitize(lon, lon_bin) - 1

    def calculate_weighted_average(threshold_km, bin_range):
        weighted_sum = 0
        total_weight = 0

        # objects from the nearest bins
        nearby_bins = df[(df['Latitude_Bin'] >= lat_bin - bin_range) & (df['Latitude_Bin'] <= lat_bin + bin_range) &
                         (df['Longitude_Bin'] >= lon_bin - bin_range) & (df['Longitude_Bin'] <= lon_bin + bin_range)]

        for i in range(len(nearby_bins)):
            lat_i, lon_i = nearby_bins.iloc[i]['Latitude'], nearby_bins.iloc[i]['Longitude']
            dist_km = great_circle((lat, lon), (lat_i, lon_i)).kilometers
            if dist_km <= threshold_km:
                weight = 1 / (dist_km + epsilon) * nearby_bins.iloc[i][weight_col]
                weighted_sum += nearby_bins.iloc[i][target_col] * weight
                total_weight += weight

        return weighted_sum, total_weight

    # Первоначальный расчет для соседних бинов (±1) и порога 500 метров
    threshold_km = primary_threshold_m / 1000
    weighted_sum, total_weight = calculate_weighted_average(threshold_km, bin_range = primary_bin_range)

    # Если нет точек в соседних бинах, увеличиваем диапазон до ±2 и порог до 1000 метров
    if total_weight == 0:
        threshold_km = secondary_threshold_m / 1000
        weighted_sum, total_weight = calculate_weighted_average(threshold_km, secondary_bin_range)  # для 500 бин 1

    if total_weight == 0:
        return np.nan # Если нет соседей, возвращаем 8 1/2, а не np.nan
    weighted_avg = weighted_sum / total_weight
    return weighted_avg

df = apartments[['Latitude', 'Longitude', 'Latitude_Bin', 'Longitude_Bin','Location Rating Roughly','Reviews Log']]
df.dropna(inplace=True)
print(df.shape)
df = df.reset_index(drop=True)


# Пример использования функции
lat = 52.489182
lon = 13.416555
weighted_rating = weighted_location_rating(lat, lon, lat_bin, lon_bin, df, weight_col = 'Reviews Log',
            target_col = 'Location Rating Roughly', primary_threshold_m=150, primary_bin_range = 1,
            secondary_threshold_m=500, secondary_bin_range = 2)
print(f"Weighted Location Rating: {weighted_rating}")

(16139, 6)
Weighted Location Rating: 9.620934607762823
